# Get started
First we need to build the project and create a nuget package.

In [1]:
# 1. Build the project
dotnet build Jowsy.DotNet.Interactive.Extensions

# Clear any older versions of this extension package from your NuGet cache
#rm ~/.nuget/packages/Jowsy.DotNet.Interactive.Extensions -Force -Recurse -ErrorAction Ignore

dotnet pack Jowsy.DotNet.Interactive.Extensions /p:PackageVersion=1.0.0

$nugets = Get-ChildItem -Recurse Jowsy.DotNet.Interactive.Extensions*.nupkg

$nugetPackage = $nugets[0].FullName

MSBuild version 17.8.3+195e7f5a3 for .NET
  Determining projects to restore...
  Restored C:\git\bim-net-interactive\src\Jowsy.CSharp\Jowsy.CSharp.csproj (in 300 ms).
  Restored C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\Jowsy.DotNet.Interactive.Extensions.csproj (in 355 ms).
  Jowsy.CSharp -> C:\git\bim-net-interactive\src\Jowsy.CSharp\bin\Debug\net8.0\Jowsy.CSharp.dll
C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\RevitConnector.cs(16,16): warning CS8618: Non-nullable event 'OnDataReceived' must contain a non-null value when exiting constructor. Consider declaring the event as nullable. [C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\Jowsy.DotNet.Interactive.Extensions.csproj]
C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\RevitConnector.cs(27,35): warning CS8625: Cannot convert null literal to non-nullable reference type. [C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\Jowsy.DotNe

Error: Command failed: SubmitCode: # 1. Build the project ...

In [2]:
#!set --value @pwsh:nugetPackage --name nugetPackage
var nugetSource = System.IO.Path.GetDirectoryName(nugetPackage);
nugetSource

C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug

In [3]:
#r "nuget:Jowsy.DotNet.Interactive.Extensions"

#i @csharp:nugetSource

Restore sources C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug Installed Packages Jowsy.DotNet.Interactive.Extensions, 1.77.0

Loading extensions from `C:\Users\sejsau\.nuget\packages\jowsy.dotnet.interactive.extensions\1.77.0\interactive-extensions\dotnet\Jowsy.DotNet.Interactive.Extensions.dll`

Revit C# Kernel Added.

## Connecting to the Revit kernel
We use the #!connect magic command to establish connection to the embedded revit kernel. Remember to start the kernel on the Revit side. Goto Addins->Net Interactive->Show Dockable Panel.

In [4]:
#!connect revit --kernel-name revit24 --revit-version 2024

Kernel added: #!revit24

If connection is established global variables are created for the current UIApplication, UIDocument and Document. Variables are shown in the kernel viewer as seen below. Those variables can be referenced in code cells chosen to run on the Revit kernel.

![](screenshot-variables.png)

In order to get IntelliSense you can import Revit API references. For the moment IntelliSense is not supported when using the revit kernel. A workaround is to uncomment the connector directive to activate the csharp-kernel.

In [8]:
#r "nuget:Revit.RevitApi.x64, 2023.0.0"
#r "nuget:Revit.RevitApiUi.x64, 2023.0.0"
using Autodesk.Revit.DB;
using Autodesk.Revit.UI;

Restore sources C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug Installed Packages Revit.RevitApi.x64, 2023.0.0 Revit.RevitApiUi.x64, 2023.0.0

## Querying the Revit Database
If you just want to make a simple query you can use display() method to output the result.

In [ ]:
#!revit24
   
   var query = new FilteredElementCollector(doc).OfClass(typeof(Wall))
                                                 .Cast<Wall>();

display(query);

NET interactive formatters will try to render presentations for objects. Objects of type Autodesk.DB.Elements are configured to be presented as HTML. 

The formatter used in Autodesk Revit is set to only recurse to a specific depth (default 3).

In the code below we return the first item of an array of walls instead of displaying. This will return it to the Revit kernel and be added to the global variables.

In [5]:
#!revit24
   
   var wall = new FilteredElementCollector(doc).OfClass(typeof(Wall))
                                                 .Cast<Wall>()
                                                 .First();

wall

Requesting value infos...Value info received...Name: uiapp, TypeName: UIApplication: Value:Autodesk.Revit.UI.UIApplication
Name: uidoc, TypeName: UIDocument: Value:Autodesk.Revit.UI.UIDocument
Name: doc, TypeName: Document: Value:Autodesk.Revit.DB.Document


In [6]:
#!revit24
display(wall.Name);

Level level = doc.GetElement(wall.LevelId) as Level;

display(level.Name);

Requesting value infos...Value info received...Name: uiapp, TypeName: UIApplication: Value:Autodesk.Revit.UI.UIApplication
Name: uidoc, TypeName: UIDocument: Value:Autodesk.Revit.UI.UIDocument
Name: doc, TypeName: Document: Value:Autodesk.Revit.DB.Document
Name: assemblyPath, TypeName: String: Value:C:\Users\sejsau\AppData\Local\Temp\revitkernel\dnxaujaw.jcp.dll
Name: wall, TypeName: Wall: Value:Autodesk.Revit.DB.Wall


Wall 1

Level 1

## Variable Sharing


In [10]:
#!revit24

//Adapted from https://thebuildingcoder.typepad.com/blog/2015/01/getting-the-wall-elevation-profile.html

// Get the external wall face for the profile
IList<Reference> sideFaces
  = HostObjectUtils.GetSideFaces(wall,
    ShellLayerType.Exterior );
 
Element e2 = doc.GetElement( sideFaces[0] );

Face face = e2.GetGeometryObjectFromReference(
  sideFaces[0] ) as Face;

// The normal of the wall external face.
XYZ normal = face.ComputeNormal( new UV( 0, 0 ) );

// Offset curve copies for visibility.
Transform offset = Transform.CreateTranslation(
  5 * normal );

// If the curve loop direction is counter-
// clockwise, change its color to RED.

Color colorRed = new Color( 255, 0, 0 );

// Get edge loops as curve loops.

IList<CurveLoop> curveLoops
  = face.GetEdgesAsCurveLoops();

// ExporterIFCUtils class can also be used for 
// non-IFC purposes. The SortCurveLoops method 
// sorts curve loops (edge loops) so that the 
// outer loops come first.
 
IList<IList<CurveLoop>> curveLoopLoop
  = ExporterIFCUtils.SortCurveLoops(
    curveLoops );



Requesting value infos...Value info received...Name: uiapp, TypeName: UIApplication: Value:Autodesk.Revit.UI.UIApplication
Name: uidoc, TypeName: UIDocument: Value:Autodesk.Revit.UI.UIDocument
Name: doc, TypeName: Document: Value:Autodesk.Revit.DB.Document
Name: assemblyPath, TypeName: String: Value:C:\Users\sejsau\AppData\Local\Temp\revitkernel\enmxeca3.kge.dll
Name: wall, TypeName: Wall: Value:Autodesk.Revit.DB.Wall
Name: curveLoops, TypeName: List`1: Value:[ [ Autodesk.Revit.DB.Line, Autodesk.Revit.DB.Line, Autodesk.Revit.DB.Line, Autodesk.Revit.DB.Line ] ]


(53,5): error CS0103: The name 'ExporterIFCUtils' does not exist in the current context


Error: Command failed: SubmitCode: #!revit24 ...